In [57]:
from tensorflow.keras import Model

In [13]:
import os
import glob
import json
import argparse
from utils.utils import calc_mean_score, save_json
from handlers.model_builder import Nima
from handlers.data_generator import TestDataGenerator

In [21]:
from os import listdir
from os.path import isfile, join

In [27]:
mypath ='D:/Amadeus/NIMA/ima_test'

In [32]:
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))

In [ ]:
# img_paths = glob.glob(os.path.join(img_dir, '*.'+img_type))


In [34]:
def image_file_to_json(img_path):
    img_dir = os.path.dirname(img_path)
    img_id = os.path.basename(img_path).split('.')[0]

    return img_dir, [{'image_id': img_id}]

In [45]:
def image_dir_to_json(img_dir, img_type='jpg'):
    img_paths = glob.glob(os.path.join(img_dir, '*.'+img_type))

    samples = []
    for img_path in img_paths:
        img_id = os.path.basename(img_path).split('.')[0]
        samples.append({'image_id': img_id})

    return samples


In [46]:
def predict(model, data_generator):
    return model.predict_generator(data_generator, workers=8, use_multiprocessing=True, verbose=1)

In [76]:
 # build model and load weights
nima = Nima("MobileNet", weights=None)
nima.build()
nima.nima_model.load_weights('weights_mobilenet_aesthetic_0.07.hdf5')

In [116]:
def get_samples(image_source):
    if os.path.isfile(image_source):
        image_dir, samples = image_file_to_json(image_source)
    else:
        image_dir = image_source
        samples = image_dir_to_json(image_dir, img_type='jpg')
    return (samples,image_dir)

In [117]:
test_samples = get_samples('D:/Amadeus/NIMA/ima_test')

In [131]:
len(test_samples[0]) # ... images 

95

In [132]:
 data_generator = TestDataGenerator(test_samples[0], test_samples[1], 64, 10, nima.preprocessing_function(),
                                       img_format='jpg')

In [139]:
data_generator

In [120]:
predictions = predict(nima.nima_model, data_generator)

2/2 [==============================] - 1s 689ms/step


In [121]:
for i, sample in enumerate(test_samples[0]):
        sample['mean_score_prediction'] = calc_mean_score(predictions[i])
        print(sample)

{'image_id': '00000001111111', 'mean_score_prediction': 5.667959948378666}
{'image_id': '1111111111111111111111111111', 'mean_score_prediction': 5.472509325021747}
{'image_id': '1930336_67026505433_3938_n', 'mean_score_prediction': 4.507409295427351}
{'image_id': '20191229_131522', 'mean_score_prediction': 3.98761538704548}
{'image_id': 'Hao Tien', 'mean_score_prediction': 5.338753280590026}
{'image_id': 'IMG-20190820-WA0002', 'mean_score_prediction': 5.241150526826004}
{'image_id': 'IMG-20190820-WA0003', 'mean_score_prediction': 5.471899754862534}
{'image_id': 'IMG-20190820-WA0004', 'mean_score_prediction': 5.35881178764248}
{'image_id': 'IMG-20190820-WA0005', 'mean_score_prediction': 5.309744029844296}
{'image_id': 'IMG-20190820-WA0006', 'mean_score_prediction': 4.297074595888262}
{'image_id': 'IMG-20190820-WA0007', 'mean_score_prediction': 5.506776186259231}
{'image_id': 'IMG-20190820-WA0008', 'mean_score_prediction': 5.382028930842353}
{'image_id': 'IMG-20190820-WA0009', 'mean_scor

In [103]:
sample

{'image_id': 'IMG_8268', 'mean_score_prediction': 5.46431887482936}

In [87]:
# model1.load_weights('weights_mobilenet_aesthetic_0.07.hdf5')

In [ ]:
tf.keras.models.load_model(
    filepath, custom_objects=None, compile=True, options=None
)

In [47]:
def main_func(base_model_name, weights_file, image_source, predictions_file, img_format='jpg'):
    # load samples
    if os.path.isfile(image_source):
        image_dir, samples = image_file_to_json(image_source)
    else:
        image_dir = image_source
        samples = image_dir_to_json(image_dir, img_type='jpg')

    # build model and load weights
    nima = Nima(base_model_name, weights=None)
    nima.build()
    nima.nima_model.load_weights(weights_file)

    # initialize data generator
    data_generator = TestDataGenerator(samples, image_dir, 64, 10, nima.preprocessing_function(),
                                       img_format=img_format)

    # get predictions
    predictions = predict(nima.nima_model, data_generator)

    # calc mean scores and add to samples
    for i, sample in enumerate(samples):
        sample['mean_score_prediction'] = calc_mean_score(predictions[i])

    print(json.dumps(samples, indent=2))

    if predictions_file is not None:
        save_json(samples, predictions_file)